Hashtag BOW
------
\#WhyImNotVotingForHillary > Why Im Not Voting For Hillary

In [28]:
import pandas as pd
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
sts_gold = pd.read_csv('../data/sts_gold_v03/sts_gold_tweet.csv', index_col='id', sep=';')

In [4]:
sts_gold.head()

,polarity,tweet
id,,
1467933112,0,the angel is going to miss the athlete this we...
2323395086,0,It looks as though Shaq is getting traded to C...
1467968979,0,@clarianne APRIL 9TH ISN'T COMING SOON ENOUGH
1990283756,0,drinking a McDonalds coffee and not understand...
1988884918,0,So dissapointed Taylor Swift doesnt have a Twi...


In [10]:
hashtagSeries = sts_gold['tweet'].apply(lambda x: re.findall(r'\#\w+', x))

In [20]:
def hashtag_splitter(hashtag_list):
    wordlist = []
    for hashtag in hashtag_list:
        token = re.findall('[A-Z][^A-Z]{1,}', hashtag.replace('\#',''))
        wordlist.extend(token)
    return ' '.join(wordlist)

In [25]:
hashtag_splitter(["#WhyImNotVotingForHillary", "#PropCaseHashtag"])

'Why Im Not Voting For Hillary Prop Case Hashtag'

In [24]:
hashtagSplitSeries = hashtagSeries.apply(lambda x: hashtag_splitter(x))

In [30]:
cv = CountVectorizer()
hashtagWords = cv.fit_transform(hashtagSplitSeries)
htColumns = ['#_' + _ for _ in cv.get_feature_names()]
hashtagSplit_df = pd.DataFrame(hashtagWords.toarray(), index=hashtagSplitSeries.index, columns=htColumns)

print(hashtagSplit_df.shape, hashtagSplit_df.values.mean(), hashtagSplit_df.values.max())

(2034, 21) 0.000538465140235 1


In [31]:
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.preprocessing import Binarizer
from sklearn.dummy import DummyClassifier

In [32]:
models = [('DUMMY', DummyClassifier(strategy='most_frequent')),
          ('mNB' , MultinomialNB()),
          ('bNB' , BernoulliNB()),
          ('svc' , SVC())]

In [34]:
for name, model in models:
    X, Y = hashtagSplit_df, (sts_gold['polarity'] == 4).ravel()
    if name == 'bNB':
        binarize = Binarizer()
        X = binarize.fit_transform(X)
    elif name == 'svc':
        ss = StandardScaler()
        X = ss.fit_transform(X.toarray())
    print(name + ":\t", cross_val_score(model, X, Y).mean())

bNB:	 0.689774121648
svc:	 0.689282477424
mNB:	 0.689773395441
DUMMY:	 0.689282477424
